# Actividad 13: Correlación de Spearman (y ejercicio: Pearson)

- **Spearman (ρ)**: no paramétrica, basada en rangos; útil con datos no normales, ordinales, relaciones monótonas y con outliers.
- **Pearson (r)**: paramétrica; mide relación **lineal** bajo supuestos más estrictos (normalidad, homocedasticidad).

A continuación evaluamos Spearman para **TA vs DIC** y, como ejercicio, implementamos Pearson.


In [1]:
import pandas as pd
from scipy import stats

CO2Data = pd.read_csv("../data/Terminos_lagoon_TA_DIC_2023_RawData.csv")
CO2Data.head()


,sample,date,estuary,area,station,layer_depth,season,chlorophy_microg_l,cond_microsiemens_cm,depth_m,...,do_mg_l,sal_psu,sp_cond_microsiemens_cm,turbidity_fnu,temp_c,latitude,longitude,dic_micromol_kg,ta_micromol_kg,dummy_data
0,CDL01S,5/3/2020,Candelaria,River,CDL01,Surface,Dry,0.36,7015.4,0.464,...,7.12,3.56,6547.7,1.47,28.740,18.55736,-91.25012,3915,3863,3685.0
1,CDL01F,5/3/2020,Candelaria,River,CDL01,Bottom,Dry,4.19,29886.1,7.792,...,4.90,16.97,27751.2,95.33,29.028,18.55722,-91.24990,3698,3685,NaN
2,CDL02S,5/3/2020,Candelaria,River,CDL02,Surface,Dry,0.92,16691.1,0.453,...,6.99,8.94,15429.1,5.50,29.283,18.61007,-91.24410,3724,3708,3708.0
3,CDL02F,5/3/2020,Candelaria,River,CDL02,Bottom,Dry,2.23,24847.4,1.261,...,6.52,13.87,23074.0,13.44,29.024,18.61005,-91.24403,3667,3992,3992.0
4,CDL03S,5/3/2020,Candelaria,River,CDL03,Surface,Dry,0.58,46341.6,0.465,...,6.24,28.06,43670.8,3.60,28.202,18.63166,-91.29359,2928,3023,3023.0


In [3]:
from scipy import stats

def test_spearman(x, y, alpha=0.05):
    # Asegura longitudes compatibles y elimina NaNs en pares
    s = pd.DataFrame({"x": x, "y": y}).dropna()
    x_, y_ = s["x"], s["y"]

    if len(x_) != len(y_):
        raise ValueError("Las variables deben tener la misma longitud.")

    rho, pval = stats.spearmanr(x_, y_)

    print(f"Spearman's rho: {rho:.3f}")
    print(f"p-value: {pval:.4f}")

    if pval < alpha:
        print(f" Relación significativa (p < {alpha:.3f})")
    else:
        print(f" Relación no significativa (p ≥ {alpha:.3f})")

    print(" Correlación positiva" if rho > 0 else " Correlación negativa")


In [4]:
test_spearman(CO2Data["ta_micromol_kg"], CO2Data["dic_micromol_kg"])


Spearman's rho: 0.838
p-value: 0.0000
 Relación significativa (p < 0.050)
 Correlación positiva


## Ejercicio: Crear una función para calcular la correlación de **Pearson**

- Usé `scipy.stats.pearsonr`.
- Consideré:
  - Eliminar pares con valores faltantes.
  - Reportar **r**, **p-valor** y una breve interpretación.


In [6]:
from scipy import stats

def test_pearson(x, y, alpha=0.05):
    # Emparejar y eliminar NaNs
    s = pd.DataFrame({"x": x, "y": y}).dropna()
    x_, y_ = s["x"], s["y"]

    if len(x_) < 3:
        raise ValueError("Se requieren al menos 3 pares de datos para Pearson.")

    r, pval = stats.pearsonr(x_, y_)

    print(f"Pearson's r: {r:.3f}")
    print(f"p-value: {pval:.4f}")

    if pval < alpha:
        print(f" Relación lineal significativa (p < {alpha:.3f})")
    else:
        print(f" Relación lineal no significativa (p ≥ {alpha:.3f})")

    tendencia = "positiva" if r > 0 else "negativa"
    magnitud = (
        "débil" if abs(r) < 0.3 else
        "moderada" if abs(r) < 0.6 else
        "fuerte"
    )
    print(f"Interpretación: correlación {tendencia} {magnitud}.")


In [7]:
# 1) TA vs DIC
test_pearson(CO2Data["ta_micromol_kg"], CO2Data["dic_micromol_kg"])

# 2) DIC vs Salinidad
test_pearson(CO2Data["dic_micromol_kg"], CO2Data["sal_psu"])


Pearson's r: 0.882
p-value: 0.0000
 Relación lineal significativa (p < 0.050)
Interpretación: correlación positiva fuerte.
Pearson's r: -0.572
p-value: 0.0000
 Relación lineal significativa (p < 0.050)
Interpretación: correlación negativa moderada.


## Conclusión comparativa: Spearman vs Pearson

En esta actividad trabajé con dos medidas de correlación:  

- Spearman (ρ):  
  Es un coeficiente no paramétrico que se basa en los rangos de los datos. Se usa cuando los datos no son normales, cuando las variables son ordinales, cuando la relación es monotónica (creciente o decreciente, pero no necesariamente lineal), y también cuando hay outliers, porque es más robusto.  

- Pearson (r):  
  Es un coeficiente paramétrico que mide la relación lineal entre dos variables cuantitativas. Sus supuestos son más estrictos: datos normales, relación lineal y homocedasticidad. Es más sensible a los outliers.  

### Comparación en este dataset
Al aplicar Spearman y Pearson a las mismas variables (por ejemplo TA vs DIC), los resultados pueden coincidir bastante si la relación es lineal y no hay muchos valores extremos. Pero si los supuestos no se cumplen, Pearson puede distorsionar la relación, mientras que Spearman sigue captando la tendencia monotónica.  

Conclusión: ambos coeficientes son útiles y se complementan. Pearson lo usaría cuando los datos cumplen con normalidad y linealidad. Spearman cuando los datos no cumplen esos supuestos o cuando la relación es más bien monotónica. En este ejercicio los puse juntos para comparar enfoques y quedarme con un panorama más completo para los análisis ambientales.  
